In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..', 'code'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import os
from FaceRecognition import FaceRecognition
import cv2 as cv
from facenet_pytorch import InceptionResnetV1, MTCNN
import torch
from PIL import Image
from torchvision import transforms
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time
from modAL.models import ActiveLearner
import matplotlib.pyplot as plt
import pickle

/media/mrj/documents/AI/FaceRecognition/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ALIGNED_LABLED_DATA = '/media/mrj/documents/AI/FaceRecognition/data/labaled_data/aligned'
EMBEDD_LABLED_DATA = '/media/mrj/documents/AI/FaceRecognition/data/labaled_data/embedding'
IMAGES_LABLED_DATA  = '/media/mrj/documents/AI/FaceRecognition/data/labaled_data/images'

ALIGNED_UNLABLED_DATA = '/media/mrj/documents/AI/FaceRecognition/data/unlabled_data/aligned'
EMEBDD_UNLABLED_DATA = '/media/mrj/documents/AI/FaceRecognition/data/unlabled_data/embedding'
IMAGES_UNLABLED_DATA  = '/media/mrj/documents/AI/FaceRecognition/data/unlabled_data/images'

In [15]:
LABLES = os.listdir(IMAGES_LABLED_DATA)
NUM_DATA_EACH_LABEL = {}
for index, label in enumerate(LABLES):
    NUM_DATA_EACH_LABEL[label] = len(os.listdir(os.path.join(IMAGES_LABLED_DATA, label)))
sorted_dict = dict(sorted(NUM_DATA_EACH_LABEL.items(), key=lambda item: item[1], reverse=False))

LABLES = []
for key, value in sorted_dict.items():
    if  30 >value > 10:
        LABLES.append(key)
    if len(LABLES) == 20:
        break
print(len(LABLES))

20


In [5]:
Face = FaceRecognition()

In [16]:
X = [] 
Y = []

In [17]:
model = InceptionResnetV1(pretrained='vggface2').eval()
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

for i, lable in enumerate(LABLES):
    print(f'\r{len(LABLES)}:{i}', end='')
    images_lable_path = os.path.join(IMAGES_LABLED_DATA, lable)
    aligned_lable_path = os.path.join(ALIGNED_LABLED_DATA, lable)
    
    for image_name in os.listdir(images_lable_path):
        image_path = os.path.join(images_lable_path, image_name)

        image_rgb = cv.imread(image_path)
        image = cv.cvtColor(image_rgb, cv.COLOR_BGR2GRAY)

        faces, boxes = Face.detect_haar(image, None)
        poses, aligned_faces = Face.pose_and_align(image_rgb, boxes)
        if aligned_faces is not None:
            for aligned_face in aligned_faces:
                
                image_rgb = cv.cvtColor(aligned_face, cv.COLOR_BGR2RGB)
                image_pil = Image.fromarray(image_rgb)
                image_tensor = transform(image_pil).unsqueeze(0)
                with torch.no_grad():
                    embdeding = model(image_tensor).squeeze().numpy()
                X.append(embdeding)
                Y.append(lable)

classifier = RandomForestClassifier()
learner = ActiveLearner(classifier, X_training=X, y_training=Y)
print()
print(accuracy_score(Y, learner.predict(X)))

20:19
1.0


In [18]:
print(learner.predict_proba(X[0].reshape(1, -1))[0])

[0.01 0.89 0.01 0.   0.   0.   0.   0.02 0.   0.   0.   0.   0.03 0.01
 0.   0.01 0.01 0.01 0.   0.  ]


In [19]:
pickle.dump(learner, open('../models/classic_online.pkl', 'wb'))

In [20]:
learner = pickle.load(open('../models/classic_online.pkl', 'rb'))

In [ ]:
#USING AUGMENTOR


In [1]:
import Augmentor

p = Augmentor.Pipeline("images/")
p.rotate(probability=0.9, max_left_rotation=25, max_right_rotation=25)
p.zoom(probability=0.3, min_factor=1.1, max_factor=1.6)
p.sample(10)

Initialised with 1 image(s) found.
Output directory set to images/output.

Processing <PIL.Image.Image image mode=RGB size=640x480 at 0x7F83888F2F20>: 100%|██████████| 10/10 [00:00<00:00, 77.57 Samples/s]


In [ ]:
import numpy as np
learner = pickle.load(open('../models/classic_online.pkl', 'rb'))
frame = cv.imread('../test/20250114011243321277.jpg')
gray_img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY) 
faces, boxes = Face.detect_haar(gray_img, None)
if len(faces) == 0:
    print("\rNo Faces", end='')
    assert False
poses, aligned_faces = Face.pose_and_align(frame, boxes)

embeddings = Face.extract_embeddings(aligned_faces)
confidences = []
for x in embeddings:
    x = x.reshape(1, -1)
    probas = learner.predict_proba(x)
    confidence = max(probas[0])
    confidences.append(confidence)
print(np.array(confidences))
for index, x in enumerate(embeddings):
    print(index)
    x = x.reshape(1, -1)
    learner.teach(x, np.array([102+index]))

confidences = []
for x in embeddings:
    x = x.reshape(1, -1)
    probas = learner.predict_proba(x)
    confidence = max(probas[0])
    confidences.append(confidence)
print(np.array(confidences))


frame = cv.imread('../test/20250114011326397435.jpg')
gray_img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY) 
faces, boxes = Face.detect_haar(frame, None)
if len(faces) == 0:
    print("\rNo Faces", end='')
    assert False
poses, aligned_faces = Face.pose_and_align(gray_img, boxes)
embeddings = Face.extract_embeddings(aligned_faces)
confidences = []
for x in embeddings:
    x = x.reshape(1, -1)
    probas = learner.predict_proba(x)
    confidence = max(probas[0])
    confidences.append(confidence)
print(np.array(confidences))

[0.14]
0


/media/mrj/documents/AI/FaceRecognition/.venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[0.66]
[0.14]


In [59]:
from Model import Model
import cv2 as cv
model = Model()



In [67]:
model.load_model()

In [71]:
print(model.evaluation(image=cv.imread('../api/static/images/8/20250114013247400888.jpg')))

([0.65], [array(['1'], dtype='<U25')])


In [68]:
print(model.evaluation(image=cv.imread('../test/20250114011243321277.jpg')))
print(model.evaluation(image=cv.imread('../test/20250114011326397435.jpg')))
model.ActiveLearning(image=cv.imread('../test/20250114011243321277.jpg'))
print(model.evaluation(image=cv.imread('../test/20250114011243321277.jpg')))
print(model.evaluation(image=cv.imread('../test/20250114011326397435.jpg')))


([0.12], [array(['Anna_Kournikova'], dtype='<U25')])
([0.14], [array(['Mike_Weir'], dtype='<U25')])
Initialised with 1 image(s) found.
Output directory set to /media/mrj/documents/AI/FaceRecognition/data/unlabled_data/images/1/output.

Processing <PIL.Image.Image image mode=RGB size=640x480 at 0x7F82300CD900>: 100%|██████████| 100/100 [00:07<00:00, 12.60 Samples/s]
/media/mrj/documents/AI/FaceRecognition/.venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


([0.93], [array(['1'], dtype='<U25')])
([0.45], [array(['1'], dtype='<U25')])
